# Baseline system of "call to action" detection model 

The notebook contains the code for the baseline system for the automatic detection of **Calls-to-action** (**subtask 1** of the shared task). A gradient boosting algorithm was chosen for classification, using sentence embeddings and a polarity score as features. The notebook covers the training of the system as well as the prediction on the test data and the evaluation. 

Executing the following two lines of code will install all necessary packages. 

In [3]:
%%writefile requirements.txt

pandas==2.2.3
spacy==3.8.2
scikit-learn==1.6.1
textblob==0.15.3
textblob-de==0.4.3
sentence-transformers==4.1.0
nltk==3.9.1
numpy==2.0.1

Overwriting requirements.txt


In [4]:
%pip install -r requirements.txt

Note: you may need to restart the kernel to use updated packages.


## 1. Importing training data

First, the training data was read in. 

In [5]:
# Reading in training data 
import pandas as pd

filename = "c2a_train.csv" # Path needs to be adjusted 
train_c2a = pd.read_csv(filename, sep=';')
train_c2a.head()

,id,description,C2A
0,874458912592533,"Wenn Du Wert drauf legst, dann erwarten wir di...",False
1,855617527810005,"Macht was ihr wollt, aber schreibt nicht ""Wir ...",False
2,1111407378897684,"Tja, wohl etwas schwierig, jetzt das Geld per ...",False
3,945847662120324,"UInd das Glaube ich nicht,,,in Hotels in Arfri...",False
4,993541110684312,und kein TTIP,False


The class distribution of the training data was analysed. 

In [6]:
# Absolute number of instances in each class 
class_counts_c2a = train_c2a["C2A"].value_counts()

# Relative number of instances in each class 
class_percent_c2a = train_c2a['C2A'].value_counts(normalize=True) * 100

# Summarise into a dataframe
class_table_c2a = pd.DataFrame({
    'Frequency': class_counts_c2a,
    'Percentage': class_percent_c2a.round(2)
})

print(class_table_c2a)

       Frequency  Percentage
C2A                         
False       6177       90.31
True         663        9.69


## 2. Data preprocessing and undersampling

Since the data is unbalanced, undersampling was applied to the training data. 

In [ ]:
from sklearn.utils import resample

minority_class = train_c2a[train_c2a['C2A'] == True]
majority_class = train_c2a[train_c2a['C2A'] == False]

majority_undersampled = resample(majority_class, replace=False, n_samples=len(minority_class), random_state=42)
train_c2a = pd.concat([minority_class, majority_undersampled])

print(train_c2a['C2A'].value_counts())

The training data was then pre-processed. This included basic cleansing steps, specifically the removal of URLs, hashtags and mentions. 

In [7]:
import re

def clean_text(text):
    url_pattern = r'https?://\S+|www\.\S+'
    mention_pattern = r'@\w+'
    hashtag_pattern = r'#\w+'
    combined_pattern = f'({url_pattern})|({mention_pattern})|({hashtag_pattern})'

    cleaned_text = re.sub(combined_pattern, '', text)
    return cleaned_text

The texts were then lemmatised and tokenised. 

In [8]:
import spacy

# Loading the Spacy pipeline for the German language 
nlp = spacy.load('de_core_news_md')

In [9]:
# Defining the lemmatisation function 
def text_lemmatize_tokenize(texts):
    lemmatized = []
    for doc in nlp.pipe(texts, batch_size = 50):
        tokens = [token.lemma_.lower() for token in doc if not token.is_punct]
        lemmatized.append(' '.join(tokens))
    return lemmatized

The two functions for removing certain tokens and for lemmatisation were applied to the training data. 

In [10]:
# Removing URLs, hashtags and mentions 
train_c2a['description'] = train_c2a['description'].apply(clean_text)
print(train_c2a.head())

                 id                                        description    C2A
0   874458912592533  Wenn Du Wert drauf legst, dann erwarten wir di...  False
1   855617527810005  Macht was ihr wollt, aber schreibt nicht "Wir ...  False
2  1111407378897684  Tja, wohl etwas schwierig, jetzt das Geld per ...  False
3   945847662120324  UInd das Glaube ich nicht,,,in Hotels in Arfri...  False
4   993541110684312                                      und kein TTIP  False


In [11]:
# Lemmatisation and tokenisation 
train_c2a['description'] = text_lemmatize_tokenize(train_c2a['description'].tolist())
print(train_c2a.head())

                 id                                        description    C2A
0   874458912592533  wenn du wert drauf legen dann erwarten wir die...  False
1   855617527810005  macht was ihr wollen aber schreiben nicht wir ...  False
2  1111407378897684  tja wohl etwas schwierig jetzt der geld per ch...  False
3   945847662120324  uind der glaube ich nicht,,,in hotel in arfrik...  False
4   993541110684312                                      und kein ttip  False


## 3. Feature Engineering

Next, the tweets from the training data were converted into a feature representation. Specifically, polarity was extracted as a feature using the Text Blob library. 

In [12]:
from textblob_de import TextBlobDE

# Function for determining the polarity of a tweet
def add_polarity(df):
    def calculate_sentiment_features(text):
        blob = TextBlobDE(text)
        return blob.sentiment.polarity

    df[['polarity']] = df['description'].apply(
        lambda x: pd.Series(calculate_sentiment_features(x))
    )

    return df

In addition, the tweets were represented as sentence embeddings using a Sentence BERT model. The feature vectors of each tweet consisted of the polarity value and the sentence embedding. 

In [13]:
from sentence_transformers import SentenceTransformer

# Function for representing tweets as embeddings 
def add_semantic_features(df):
    sentence_model = SentenceTransformer('distiluse-base-multilingual-cased-v2')
    texts = df['description'].astype(str).values.tolist()

    embeddings = sentence_model.encode(texts, show_progressbar=True)
    embeddings_df = pd.DataFrame(embeddings, columns=[f'embedding_{i}' for i in range(embeddings.shape[1])])

    df = pd.concat([df.reset_index(drop=True), embeddings_df.reset_index(drop=True)], axis=1)
    return df

c:\Users\felser\AppData\Local\anaconda3\envs\germeval2025\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The features were extracted from the tweets in the training data. 

In [14]:
import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to \\na2.hs-
[nltk_data]     mittweida.de\felser\Wappscfg\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to \\na2.hs-
[nltk_data]     mittweida.de\felser\Wappscfg\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt_tab to \\na2.hs-
[nltk_data]     mittweida.de\felser\Wappscfg\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [15]:
# Extraction of the polarity score
train_c2a = add_polarity(train_c2a)
print(train_c2a.head())

                 id                                        description    C2A  \
0   874458912592533  wenn du wert drauf legen dann erwarten wir die...  False   
1   855617527810005  macht was ihr wollen aber schreiben nicht wir ...  False   
2  1111407378897684  tja wohl etwas schwierig jetzt der geld per ch...  False   
3   945847662120324  uind der glaube ich nicht,,,in hotel in arfrik...  False   
4   993541110684312                                      und kein ttip  False   

   polarity  
0       0.0  
1      -1.0  
2      -0.7  
3       0.0  
4       0.0  


In [16]:
# Extraction of embedding features 
train_c2a = add_semantic_features(train_c2a)
print(train_c2a.head())

                 id                                        description    C2A  \
0   874458912592533  wenn du wert drauf legen dann erwarten wir die...  False   
1   855617527810005  macht was ihr wollen aber schreiben nicht wir ...  False   
2  1111407378897684  tja wohl etwas schwierig jetzt der geld per ch...  False   
3   945847662120324  uind der glaube ich nicht,,,in hotel in arfrik...  False   
4   993541110684312                                      und kein ttip  False   

   polarity  embedding_0  embedding_1  embedding_2  embedding_3  embedding_4  \
0       0.0    -0.017523    -0.010992     0.005523    -0.033231     0.023532   
1      -1.0    -0.036740     0.016144    -0.005617    -0.045360     0.074146   
2      -0.7     0.010182    -0.035717     0.035839    -0.001503     0.001591   
3       0.0     0.015459    -0.009660     0.053281    -0.001198     0.011009   
4       0.0    -0.001756    -0.023953    -0.005758    -0.019863    -0.000644   

   embedding_5  ...  embedding_5

## 4. Training the model 

A gradient boosting classifier was then trained using the extracted features. 

In [17]:
# Restrict training data set to features 
X_train = train_c2a.drop(columns=['id', 'description', 'C2A'])
# Extract labels from training data 
y_train = train_c2a['C2A']

In [18]:
# Training the model 
from sklearn.ensemble import GradientBoostingClassifier
model = GradientBoostingClassifier(random_state=42)
model.fit(X_train, y_train)

GradientBoostingClassifier(random_state=42)

## 5. Prediction on the test data 

Now the trained classification model could be used to make a prediction for the test data. To do this, the test data was preprocessed in the same way as the training data. 

In [19]:
# Importing the test data
filename = "c2a_test.csv" # Path needs to be adjusted 
test_c2a = pd.read_csv(filename, sep=';')

In [20]:
# Removing URLs, hashtags and mentions 
test_c2a['description'] = test_c2a['description'].apply(clean_text)

# Lemmatisation and tokenisation 
test_c2a['description'] = text_lemmatize_tokenize(test_c2a['description'].tolist())
print(test_c2a.head())

                 id                                        description
0   862053823833042              der elite meinen es nicht gut mit uns
1   855165061188585  woher denn auch der biographie von martin luth...
2  1111354418902980  wir sein der volk \r\n\r\n wie medium berichte...
3   945846622120428  haben doch was ein schwarzafikaner zusammen mi...
4   985365271501896  auf der straße mit der pack sauber so muss es ...


Subsequently, the polarity score was also determined for the tweets in the test data and they were converted into sentence embeddings. 

In [21]:
# Extraction of the polarity score
test_c2a = add_polarity(test_c2a)

# Extraction of embedding features 
test_c2a = add_semantic_features(test_c2a)
print(test_c2a.head())

                 id                                        description  \
0   862053823833042              der elite meinen es nicht gut mit uns   
1   855165061188585  woher denn auch der biographie von martin luth...   
2  1111354418902980  wir sein der volk \r\n\r\n wie medium berichte...   
3   945846622120428  haben doch was ein schwarzafikaner zusammen mi...   
4   985365271501896  auf der straße mit der pack sauber so muss es ...   

   polarity  embedding_0  embedding_1  embedding_2  embedding_3  embedding_4  \
0     -0.50    -0.014092     0.003256     0.014165    -0.018139    -0.008796   
1      0.00    -0.011190    -0.026479    -0.077471    -0.039432     0.048622   
2     -0.35    -0.024417     0.005405    -0.013448     0.001764     0.058810   
3      1.00     0.010520     0.014296     0.020480    -0.009629     0.024485   
4      0.00    -0.018551     0.006529     0.021011     0.002873     0.001395   

   embedding_5  embedding_6  ...  embedding_502  embedding_503  embedding_

The feature representation was passed to the gradient boosting model to make a prediction. 

In [22]:
# Restrict test data set to features 
X_test = test_c2a.drop(columns=['id', 'description'])
y_test_pred = model.predict(X_test)

## 6. Evaluation of results 

The predictions based on the test data were compared with the gold standard and evaluation metrics were calculated. The results achieved serve as a guide and baseline for the competition participants. 

In [23]:
# Importing the gold standard 
filename = "c2a_gold.csv" # Path needs to be adjusted 
gold_c2a = pd.read_csv(filename, sep=';')

In [24]:
# Check that the IDs from the test data and the gold standard are in the same order. 
gold_c2a["id"].tolist() == test_c2a["id"].tolist()

True

In [25]:
# The true label of the tweets was extracted from the gold standard. 
y_true = gold_c2a['C2A']

The macro F1 measure is used as the evaluation metric for the ranking in the competition leaderboard. In addition, other evaluation metrics such as precision and recall are calculated for the individual classes, as well as the macro and weighted average. 

In [26]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import numpy as np
test_report = classification_report(y_true, y_test_pred)
print("Test Classification Report:")
print(test_report)

Test Classification Report:
              precision    recall  f1-score   support

       False       0.91      0.99      0.95      2693
        True       0.60      0.13      0.21       289

    accuracy                           0.91      2982
   macro avg       0.76      0.56      0.58      2982
weighted avg       0.88      0.91      0.88      2982

